# Import libs und Daten

In [22]:
import pandas as pd
from datetime import datetime
import numpy as np
# XML-Lib laden
import xml.etree.ElementTree as ET
from datawrapper import Datawrapper
dw = Datawrapper(access_token = "79263c731ee5ea069033d6c0e7f62798e440a05b21a43a4b1f1ae765a7eab433")
from datetime import datetime
from datetime import date
import random
import os
import glob
os.getcwd()
os.chdir(r"C:\\Users\\c_knoop\\Desktop\\BTW_2021")

In [24]:
#url_xml = "./analyse_alt/rohdaten/wahl_gemeinden.xml"
url_xml = "./test_daten/F.210906.124700.GE.xml"
#url_xml = "./test_daten/F.210906.132800.WK.xml"
root = ET.parse(url_xml).getroot()
datenstand = "25.09.2021, 17:24 Uhr"


# historisches Wahlergebnis

## DAten einlesen und auf notwendige Columns begrenzen

In [25]:
wahlergebnisse_2017 = pd.read_excel("./data/BW17_Ergebnisse_ORIGINALE.xlsx", sheet_name="BW17_Ergebnisse_GE_TG")

In [26]:
wahlergebnisse_2017.columns

Index(['Wahl', 'WK-Nr', 'WK-Name', 'Ebene', 'AGS', 'Ortname',
       'Briefwahl_Sonderfall', 'Wahlberechtigte', 'Wähler', 'ungültige_1',
       'gültige_1', 'CDU_1', 'DIE LINKE_1', 'SPD_1', 'AfD_1', 'GRÜNE_1',
       'NPD_1', 'FDP_1', 'PIRATEN_1', 'FREIE WÄHLER_1', 'BüSo_1', 'MLPD_1',
       'ÖDP_1', 'Die PARTEI_1', 'Bürgerkandidat_1', 'Der Auserwählte_1',
       'Heimat Region Zwickau_1', 'Held_1', 'Hoyer_1',
       'Internationalistisch. Bündnis_1', 'RINK_1', 'SGP_1',
       'Wir sind LEIPZIGER_1', 'ungültige_2', 'gültige_2', 'CDU_2',
       'DIE LINKE_2', 'SPD_2', 'AfD_2', 'GRÜNE_2', 'NPD_2', 'FDP_2',
       'PIRATEN_2', 'FREIE WÄHLER_2', 'BüSo_2', 'MLPD_2', 'BGE_2', 'DiB_2',
       'ÖDP_2', 'Die PARTEI_2', 'Tierschutzpartei_2', 'V-Partei³_2'],
      dtype='object')

In [27]:
df_hist = wahlergebnisse_2017[['WK-Nr', 'WK-Name', 'AGS', 'Ortname',
       'Wahlberechtigte', 'Wähler', 'ungültige_1',
       'gültige_1', 'CDU_1', 'DIE LINKE_1', 'SPD_1', 'AfD_1', 'GRÜNE_1',
       'NPD_1', 'FDP_1', 'PIRATEN_1', 'FREIE WÄHLER_1', 'BüSo_1', 'MLPD_1',
       'ÖDP_1', 'Die PARTEI_1', 'Bürgerkandidat_1', 'Der Auserwählte_1',
       'Heimat Region Zwickau_1', 'Held_1', 'Hoyer_1',
       'Internationalistisch. Bündnis_1', 'RINK_1', 'SGP_1',
       'Wir sind LEIPZIGER_1', 'ungültige_2', 'gültige_2', 'CDU_2',
       'DIE LINKE_2', 'SPD_2', 'AfD_2', 'GRÜNE_2', 'NPD_2', 'FDP_2',
       'PIRATEN_2', 'FREIE WÄHLER_2', 'BüSo_2', 'MLPD_2', 'BGE_2', 'DiB_2',
       'ÖDP_2', 'Die PARTEI_2', 'Tierschutzpartei_2', 'V-Partei³_2']]

In [32]:
# richtige Formate verteilen
df_hist = df_hist.replace("x", "0")
# using dictionary to convert specific columns
convert_dict = {'WK-Nr': str,
                'AGS': str, 
                "AfD_1": int, 
                "NPD_1": int, 
                "PIRATEN_1": int, 
                "BüSo_1": int,
                "MLPD_1": int, 
                "ÖDP_1": int, 
                "Die PARTEI_1": int, 
                "Bürgerkandidat_1": int, 
                "Der Auserwählte_1": int, 
                "Heimat Region Zwickau_1": int, 
                "Held_1": int, 
                "Hoyer_1": int, 
                "Internationalistisch. Bündnis_1": int, 
                "RINK_1": int, 
                "SGP_1": int, 
                "Wir sind LEIPZIGER_1": int, 
                "FREIE WÄHLER_1": int
               }
 
    
    
df_hist = df_hist.astype(convert_dict)

# rename columns
df_hist.rename(columns={"WK-Nr": "KreisId", "WK-Name":"KreisName", "Ortname": "Ort"}, inplace=True)

KeyError: 'Only a column name can be used for the key in a dtype mappings argument.'

## Städte zusammenführen

In [40]:
# Städte zusammenfassen
# Stadtname und übergreifende AGS
city_dict = {"Leipzig":"14713000", "Dresden":"14612000"}

# Ergebnisliste

def group_cities_results(df, city_dict): 
   
    list_cities = list(city_dict.keys())
    df_final = df[~df["Ort"].str.contains("|".join(list_cities))]
    
    print(df_final.shape)
    for city in city_dict:
        df_ = df[df["Ort"].str.contains(city)]
        
        # Ort und AGS vereinheitlichen
        df_["Ort"] = city
        df_["AGS"] = city_dict[city]
        df_["KreisId"] = df_["Ort"].map(lambda x: "152/153" if x == "Leipzig" else "159/160")
        df_["KreisName"] = df_["Ort"].map(lambda x: "Leipzig I/II" if x == "Leipzig" else "Dresden I/DD-Bautzen II")
        # über neuen, einheitlichen Ortsnamen gruppieren
        df_group = df_.groupby(['KreisId', 'KreisName', 'AGS', 'Ort',], as_index=False).sum()
       
        # Daten wieder zusammenführen
        df_final = df_final.append(df_group)
       
    
    return df_final
    
df_2017 = group_cities_results(df_hist, city_dict)    
df_2017_erststimme = df_hist.copy()

(420, 49)


<ipython-input-40-ac5dbdb3b36e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["Ort"] = city
<ipython-input-40-ac5dbdb3b36e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["AGS"] = city_dict[city]
<ipython-input-40-ac5dbdb3b36e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [41]:
df_2017[df_2017["Ort"].str.contains("Leipzig")]

,KreisId,KreisName,AGS,Ort,Wahlberechtigte,Wähler,ungültige_1,gültige_1,CDU_1,DIE LINKE_1,...,PIRATEN_2,FREIE WÄHLER_2,BüSo_2,MLPD_2,BGE_2,DiB_2,ÖDP_2,Die PARTEI_2,Tierschutzpartei_2,V-Partei³_2
0,152/153,Leipzig I/II,14713000,Leipzig,448372,336605,3188,333417,86697,75080,...,1817,2745,440,505,2051,1654,706,7408,5432,832


## Historische Daten Erststimme vorbereiten

In [10]:
# hier geht es darum, den stärksten Kandidaten für die Gemeinde 2017 herauszusuchen... ist optional, lassen wir erstmal 

In [11]:
# def create_hist_erststimme(df): 
    
#     all_kreis_id = df.KreisId.unique(): 
    


# df_2017_es = create_hist_erststimme(df_2017)

## Historische Daten Zweitstimme

Hier benötige ich die **Wahlbeteiligung und die Prozentzahl pro Partei** bzw. sonstige

In [42]:
def create_hist_zweitstimme(df):
    
    # neues Dataframe für historische Zweitstimmen
    cols = ["AGS", "Wahlbeteiligung", "2017_AfD_%", "2017_CDU_%", "2017_LINKE_%","2017_SPD_%", \
           "2017_GRÜNE_%", "2017_FDP_%", "2017_sonstige_%"]
    
    new_df = pd.DataFrame(columns=cols)
    
    
    df = df[['KreisId', 'KreisName', 'AGS', 'Ort', 'Wahlberechtigte', 'Wähler', \
            'gültige_2', 'CDU_2','DIE LINKE_2', 'SPD_2', 'AfD_2', 'GRÜNE_2', 'NPD_2', 'FDP_2',\
       'PIRATEN_2', 'FREIE WÄHLER_2', 'BüSo_2', 'MLPD_2', 'BGE_2', 'DiB_2',\
       'ÖDP_2', 'Die PARTEI_2', 'Tierschutzpartei_2', 'V-Partei³_2']]
    
    all_ags = df.AGS.unique()
        
    for ags in all_ags: 
        df_ = df[df.AGS==ags]
        
        wbt = round(df_["Wähler"] / df_["Wahlberechtigte"]  * 100, 1).item()
        afd = round(df_.AfD_2 / df_.gültige_2 * 100, 1).item()
        cdu = round(df_.CDU_2 / df_.gültige_2 * 100, 1).item()
        linke = round(df_["DIE LINKE_2"] / df_.gültige_2 * 100, 1).item()
        spd = round(df_.SPD_2 / df_.gültige_2 * 100, 1).item()
        grüne = round(df_.GRÜNE_2 / df_.gültige_2 * 100, 1).item()
        fdp = round(df_.FDP_2 / df_.gültige_2 * 100, 1).item()
        sonstige_stimmen = df_.gültige_2.item() - (df_.AfD_2 + df_.CDU_2 +df_["DIE LINKE_2"] + df_.SPD_2 + df_.GRÜNE_2 + df_.FDP_2).item()
        sonstige = round(sonstige_stimmen/df_.gültige_2*100,1).item()
        
        ser_data = [ags, wbt, afd, cdu, linke, spd, grüne, fdp, sonstige]
        new_ser = pd.Series(ser_data, index=cols)
        
        new_df = new_df.append(new_ser, ignore_index=True)
    
    return new_df
        


df_2017_zw = create_hist_zweitstimme(df_2017)

# Herauslesen Daten Dataframe long

In [43]:
#HELPER
# Ordnet die Beschreibung des Tags <Gruppenergebnis> einer Klarbzeichung zu
# diese Beschreibung ist unter dem Attribut Position abgespeichert

# Check ob sich in einem String eine Nummer befindet
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


In [44]:
# Dict Parteien 

In [45]:
# Offizielle Listennummer mit Parteiname BTW 2021
parteien_dict = {"1":"AfD", "2":"CDU", "3":"Die Linke", "4":"SPD", "5":"FDP", "6":"Grüne", \
                # kleinere Parteien (sonstige)
                "7":"Tierschutzpartei", "8":"Die Partei", "9":"NPD", "10":"Freie Wähler", \
                 "11":"PIRATEN", "12":"ÖDP", "13":"V-Partei³", "14":"MLPD", "15":"dieBasis", 
                 "16": "Bündnis C", "17": "III. Weg", "18":"DKP", "19":"Die Humanisten", \
                 "20":"Gesundheitsforschung", "21":"Team Todenhöfer", "22":"Volt", \
                 # kleinere Parteien ohne Landesliste
                 "23": "ASMUSfddB", "24":"AufrechterDemokrat", "25":"Brähmig", "26":"BüSo", \
                 "27":"Daniel Richter", "28":"Direktkandidat Daniel Zimmet", "29":"Direktkandidat Frank Hannig", \
                 "30": "DK Schöllner", "31":"Ein Leipziger für Leipzig", "32":"#KLIMAfirst", \
                 "33": "Internationalistisches Bündnis", "34": "Internationalistisches Bündnis", \
                 "35": "Internationalistisches Bündnis", "36":"Ischgl", "37":"KARLKOMMT", \
                 "38": "LKR", "39": "Marcus Fuchs - Querdenken 351", "40":"Queck", "41":"REICHEL",\
                 "42":"Röhder", "43":"UNO - Menschrecht - FRIEDEN für Alle", "44":"WIR sind LEIPZIGER", \
                 "45":"www.zeitkaufhaus.de Martin Bayer", \
                 # Bezeichner für gültige und ungültige Stimmen
                "GUE": "gültige Stimmen", "UNG": "Ungültige Stimmen"

                }

In [46]:


# jedes elem in root ist der Tag <Gebietsergebnis>, in dem immer eine komplette Gemeinde steckt. 
def read_xml(root):
    
    # Variablen in Datensatz
    erststimmen = "DIREKT"
    zweitstimmen = "LISTE"


    
    # zwei Dataframes werden erstellt, eins für die Ergebnisse der Parteien, 
    # ein anderes für Wahlstatistiken etc. 
    
    # DF Ergebnisse
    cols_result = ["AGS", "KreisId", "Ort", "Partei", "ParteiId", "Stimmart", "Stimmen", "Anteil"]
    df_results = pd.DataFrame(columns=cols_result)
    
    # DF Statistiken
    col_stats = ["AGS", "Ort", "WKNr", "Statistik", "Stimmen", "Anteil"]
    df_stats = pd.DataFrame(columns=col_stats)
    
    # Durchlauf durch alle <Gruppenergebnis>, sammeln der Gemeindedaten
    for elem in root: 
        ags = elem.attrib.get("GemNr")
        name = elem.attrib.get("GemName") 
        wk_nr = elem.attrib.get("KreisNr")
        # Unterpunkt ist <Gruppenergebnis>, hier findet sich unter "Position" die Beschreibung des jew. Datensatzes
        for sub in elem: 
            position = sub.attrib.get("Position")
            
            # Check ob position eine Nummer ist - dann handelt es sich nämlich um ein Partei-Stimmergebnis
            # dann wird das result-Dataframe befüllt
            if (has_numbers(position)) | (position == "UNG") | (position == "GUE"): 
                partei = parteien_dict[position]
                for node in sub: 
                    kind_of_votes = node.attrib.get("Stimmart")
                    if kind_of_votes == erststimmen: 
                        stimmart = "Erststimme"
                    else: 
                        stimmart = "Zweitstimme"
                    count_votes = node.attrib.get("Stimmen")
                    votes_share = node.attrib.get("Anteil")
                    ser_results = pd.Series([ags, wk_nr, name, partei, position, stimmart, count_votes, votes_share], index=cols_result)
                    df_results = df_results.append(ser_results, ignore_index=True)
            
            # wenn Buchstaben-Kennung wird das statistische Dataframe befüllt
            else:
                
                info_node = sub[0]
                stimmen = info_node.attrib.get("Stimmen")
                anteil = info_node.attrib.get("Anteil")
                
                if position == "WB": 
                    statistik = "Wahlberechtigte"
                    anteil = "100"
                elif position == "WAE":
                    statistik = "Wähler"
                else: 
                    print("Fehler")
               
                ser_stats = pd.Series([ags, name, wk_nr, statistik, stimmen, anteil], index=col_stats)
                df_stats = df_stats.append(ser_stats, ignore_index=True)
        
    return df_results, df_stats
                    
                    
            #if position
            
           

            
            
            
df_results_abfrage, df_daten_abfrage = read_xml(root)

## ACHTUNG: Nur vor der Wahl - alle Werte auf Null!

## richtige Datentypen vergeben

In [47]:
# richtige Datentypen vergeben
df_results_abfrage["Stimmen"] = df_results_abfrage["Stimmen"].astype(int)
df_results_abfrage["Anteil"] = df_results_abfrage["Anteil"].astype(float)
df_results_abfrage["KreisId"] = df_results_abfrage["KreisId"].astype(str)
df_results_abfrage["ParteiId"] = df_results_abfrage["ParteiId"].astype(str)

df_daten_abfrage["Stimmen"] = df_daten_abfrage["Stimmen"].astype(int)
df_daten_abfrage["Anteil"] = df_daten_abfrage["Anteil"].astype(float)

In [48]:
df_results_abfrage

,AGS,KreisId,Ort,Partei,ParteiId,Stimmart,Stimmen,Anteil
0,14730010,151,Arzberg,Ungültige Stimmen,UNG,Erststimme,7,0.9
1,14730010,151,Arzberg,Ungültige Stimmen,UNG,Zweitstimme,8,1.0
2,14730010,151,Arzberg,gültige Stimmen,GUE,Erststimme,793,99.1
3,14730010,151,Arzberg,gültige Stimmen,GUE,Zweitstimme,792,99.0
4,14730010,151,Arzberg,AfD,1,Erststimme,198,25.0
...,...,...,...,...,...,...,...,...
15414,14523460,166,Werda,Gesundheitsforschung,20,Zweitstimme,0,0.0
15415,14523460,166,Werda,Team Todenhöfer,21,Zweitstimme,0,0.0
15416,14523460,166,Werda,Volt,22,Zweitstimme,0,0.0
15417,14523460,166,Werda,ASMUSfddB,23,Erststimme,0,0.0


## Aus mehreren Stadt-Ergebnissen ein Ergebnis und einen Datensatz für jede Großstadt

In [50]:
# Stadtname und übergreifende AGS
city_dict = {"Leipzig":"14713000", "Dresden":"14612000"}

# Ergebnisliste

def group_cities_results(df, city_dict): 
    df_final = df.copy()
    
    for city in city_dict:
        df_ = df[df["Ort"].str.contains(city)]
        # Ort und AGS vereinheitlichen
        df_["Ort"] = city
        df_["AGS"] = city_dict[city]
        
        # über neuen, einheitlichen Ortsnamen gruppieren
        df_group = df_.groupby(["AGS", "Ort", "Partei", "ParteiId", "Stimmart"], as_index=False).agg({"Stimmen":"sum", \
                                                                                                     "KreisId":"first"})
        
        
        # Anteile der Parteien neu ausrechnen
        # Erststimmen
        df_group.loc[df_group["Stimmart"]=="Erststimme", "Anteil"] = round(df_group.loc[df_group["Stimmart"]=="Erststimme"]["Stimmen"] / \
                                                    df_group.loc[(df_group["Stimmart"]=="Erststimme") & \
                                                             (df_group["Partei"]=="gültige Stimmen")]["Stimmen"]\
                                                                       .item() * 100,1)
        #Zweitstimmen
        df_group.loc[df_group["Stimmart"]=="Zweitstimme", "Anteil"] = round(df_group.loc[df_group["Stimmart"]=="Zweitstimme"]["Stimmen"] / \
                                                    df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & \
                                                             (df_group["Partei"]=="gültige Stimmen")]["Stimmen"]\
                                                                       .item() * 100,1)
        
        # gültige Erststimmen
        df_group.loc[(df_group["Stimmart"]=="Erststimme") & (df_group["Partei"]=="gültige Stimmen"), "Anteil"] = round(\
            df_group.loc[(df_group["Stimmart"]=="Erststimme") & (df_group["Partei"]=="gültige Stimmen")]["Stimmen"].item() /\
            (df_group.loc[(df_group["Stimmart"]=="Erststimme") & (df_group["Partei"]=="gültige Stimmen")]["Stimmen"].item() + \
            df_group.loc[(df_group["Stimmart"]=="Erststimme") & (df_group["Partei"]=="Ungültige Stimmen")]["Stimmen"].item()) \
            * 100, 1) 
        # gültige Zweitstimmen
        df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & (df_group["Partei"]=="gültige Stimmen"), "Anteil"] = round(\
            df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & (df_group["Partei"]=="gültige Stimmen")]["Stimmen"].item() /\
            (df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & (df_group["Partei"]=="gültige Stimmen")]["Stimmen"].item() + \
            df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & (df_group["Partei"]=="Ungültige Stimmen")]["Stimmen"].item()) \
            * 100, 1) 
        
        df_final = df_final[~df_final["Ort"].str.contains(city)]
        df_final = df_final.append(df_group)
    
    return df_final
    

#df_ergebnisse = group_cities_results(df_results_abfrage, city_dict) 
df_ergebnisse = df_results_abfrage.copy()

In [51]:
# Datenliste

def group_cities_data(df, city_dict): 
    # Kopie anfertigen, aus der später die einzelnen Datensätze entfernt und angehangen werden. 
    df_final = df.copy()
    
    for city in city_dict:
        df_ = df[df["Ort"].str.contains(city)]
        # Ort und AGS vereinheitlichen
        df_["Ort"] = city
        df_["AGS"] = city_dict[city]
        # Wahlkreis-Nummern herausfinden
        min_wk = df_["WKNr"].astype(int).min()
        max_wk =  df_["WKNr"].astype(int).max()
        df_["WKNr"] = f"{min_wk}-{max_wk}"
        
        # über neuen, einheitlichen Ortsnamen gruppieren
        df_group = df_.groupby(["AGS", "Ort", "WKNr", "Statistik"], as_index=False).agg({"Stimmen":"sum"})
        
        
        # Anteile der Parteien neu ausrechnen
        # Erststimmen
        df_group.loc[df_group["Statistik"]=="Wahlberechtigte", "Anteil"] = 100
        #Zweitstimmen
        df_group.loc[df_group["Statistik"]=="Wähler", "Anteil"] = round(df_group.loc[df_group["Statistik"]=="Wähler"]["Stimmen"] / \
                                                    df_group.loc[df_group["Statistik"]=="Wahlberechtigte"]["Stimmen"].item() * 100,1)
        
       
        df_final = df_final[~df_final["Ort"].str.contains(city)]
        df_final = df_final.append(df_group)
    
    return df_final

# try: 
#     df_daten = group_cities_data(df_daten_abfrage, city_dict)
# except: 
#     df_daten = df_daten_abfrage.copy()
    
df_daten = df_daten_abfrage.copy()

## Daten in Erst- und Zweitstimme teilen

In [52]:
# Daten Ergebnisse aufsplitten und zusätzlich eine "andere Parteien"-Version ausspielen

def split_up_data(df_ergebnisse): 
    df_ergebnisse_es = df_ergebnisse[df_ergebnisse["Stimmart"]=="Erststimme"]
    df_ergebnisse_zs = df_ergebnisse[df_ergebnisse["Stimmart"]=="Zweitstimme"]
    return df_ergebnisse_es, df_ergebnisse_zs

df_es, df_zs = split_up_data(df_ergebnisse)

# ZWEITSTIMME 

## "Andere Parteien" einrichten

In [156]:
df_es

,AGS,KreisId,Ort,Partei,ParteiId,Stimmart,Stimmen,Anteil
0,14730010,151,Arzberg,Ungültige Stimmen,UNG,Erststimme,7,0.9
2,14730010,151,Arzberg,gültige Stimmen,GUE,Erststimme,793,99.1
4,14730010,151,Arzberg,AfD,1,Erststimme,198,25.0
6,14730010,151,Arzberg,CDU,2,Erststimme,98,12.4
8,14730010,151,Arzberg,Die Linke,3,Erststimme,100,12.6
...,...,...,...,...,...,...,...,...
30,14612000,159,Dresden,SPD,4,Erststimme,65702,24.9
34,14612000,159,Dresden,Ungültige Stimmen,UNG,Erststimme,2691,1.0
38,14612000,159,Dresden,dieBasis,15,Erststimme,3009,1.1
40,14612000,159,Dresden,gültige Stimmen,GUE,Erststimme,263882,99.0


In [157]:
def reduce_number_of_parties(df): 
    liste_standing_terms = ["Ungültige Stimmen", "gültige Stimmen", "CDU", "Die Linke", \
                           "SPD", "AfD", "Grüne", "FDP"]

    
    try: 
        sti = df.Stimmart.iloc[0]
    except: 
        sti = "keine Datensätze gefunden"

    df_ = df[~df["Partei"].isin(liste_standing_terms)]
    df_rest = df[df["Partei"].isin(liste_standing_terms)]
    df_flat = df_.groupby(["AGS", "Ort"], as_index=False).agg({"Stimmen":"sum", "Anteil":"sum"})




    if sti == "Erststimme": 
        df_flat["Stimmart"] = "Erststimme"
        df_flat["Partei"] = "andere Parteien"
        df_flat = df_flat[['AGS', 'Ort', 'Partei', 'Stimmart', 'Stimmen', 'Anteil']]
        df_flat = df_flat.append(df_rest)
    elif sti == "Zweitstimme": 
        df_flat["Stimmart"] = "Zweitstimmen"
        df_flat["Partei"] = "andere Parteien"
        df_flat = df_flat[['AGS', 'Ort', 'Partei', 'Stimmart', 'Stimmen', 'Anteil']]
        df_flat = df_flat.append(df_rest)
    else: 
        df_flat["Stimmart"] = ""
        df_flat["Partei"] = ""
        df_flat = df_flat[['AGS', 'Ort', 'Partei', 'Stimmart', 'Stimmen', 'Anteil']]
        print('keine Stimmen gefunden')




    return df_flat

    
    
    
    
df_zs_flat = reduce_number_of_parties(df_zs)    

In [158]:
df_zs_flat

,AGS,Ort,Partei,Stimmart,Stimmen,Anteil,KreisId,ParteiId
0,14511000,"Chemnitz, Stadt",andere Parteien,Zweitstimmen,15536,11.9,NaN,NaN
1,14521010,Amtsberg,andere Parteien,Zweitstimmen,269,12.6,NaN,NaN
2,14521020,"Annaberg-Buchholz, Stadt",andere Parteien,Zweitstimmen,1946,15.9,NaN,NaN
3,14521035,"Aue-Bad Schlema, Stadt",andere Parteien,Zweitstimmen,2023,16.6,NaN,NaN
4,14521040,Auerbach,andere Parteien,Zweitstimmen,286,16.5,NaN,NaN
...,...,...,...,...,...,...,...,...
17,14612000,Dresden,FDP,Zweitstimme,26112,9.9,159,5
23,14612000,Dresden,Grüne,Zweitstimme,22965,8.7,159,6
31,14612000,Dresden,SPD,Zweitstimme,65880,25.0,159,4
35,14612000,Dresden,Ungültige Stimmen,Zweitstimme,2664,1.0,159,UNG


## Daten auf wide umstellen / Prefix vergeben

In [159]:
def results_long_to_wide(df_ergebnisse ,prefix="2_"): 
    df_results_wide = pd.pivot(df_ergebnisse, index=["AGS", "Ort"], columns="Partei", values=["Stimmen", "Anteil"])\
        .reset_index().fillna("0")
   
    # Multi-Index-Columns werden aufgelöst
    df_results_wide.columns = [" ".join(col).strip() for col in df_results_wide.columns.values]
   
    
    # Columns erhalten ein Präfix
    # Präfixhinzufügen
    df_results_wide = df_results_wide.add_prefix(prefix)
   

    # Columns AgS und Ort Prefix entfernen
    df_results_wide.rename(columns={"1_AGS":"AGS", "1_Ort":"Ort"}, inplace=True)
    df_results_wide.rename(columns={"2_AGS":"AGS", "2_Ort":"Ort"}, inplace=True)
   

    df_results_wide.fillna(0)
    
    # convert Stimmen from float to int
    filter_stimmen = [col for col in df_results_wide if (col.startswith("1_Stimmen")) | (col.startswith("2_Stimmen")) ]
    df_results_wide[filter_stimmen] = df_results_wide[filter_stimmen].astype(int)
    
    return df_results_wide
 
df_results = results_long_to_wide(df_zs_flat)   

## Korrekter Anteil für "andere Parteien" berechnen

In [160]:
def correct_anteil_andere_parteien(df_re):
    
    # erst Anteil für Erststimmen andere Parteinen
    try: 
        df_re["1_Anteil andere Parteien"] = round(df_re["1_Stimmen andere Parteien"] / df_re["1_Stimmen gültige Stimmen"] * 100,1)
        df_re["1_Anteil andere Parteien"] = df_re["1_Anteil andere Parteien"].astype(float)
        
    except:
        try:
            
            df_re["2_Anteil andere Parteien"] = round(df_re["2_Stimmen andere Parteien"] / df_re["2_Stimmen gültige Stimmen"] * 100,1)
            df_re["2_Anteil andere Parteien"] = df_re["2_Anteil andere Parteien"].astype(float)
    
        except: 
            print("Fehler!")
   

    return df_re

df_results_correct = correct_anteil_andere_parteien(df_results)


In [162]:
df_results_correct

,AGS,Ort,2_Stimmen AfD,2_Stimmen CDU,2_Stimmen Die Linke,2_Stimmen FDP,2_Stimmen Grüne,2_Stimmen SPD,2_Stimmen Ungültige Stimmen,2_Stimmen andere Parteien,2_Stimmen gültige Stimmen,2_Anteil AfD,2_Anteil CDU,2_Anteil Die Linke,2_Anteil FDP,2_Anteil Grüne,2_Anteil SPD,2_Anteil Ungültige Stimmen,2_Anteil andere Parteien,2_Anteil gültige Stimmen
0,14511000,"Chemnitz, Stadt",32715,16357,16381,10936,6107,32674,1286,15536,130706,25.0,12.5,12.5,8.4,4.7,25.0,1.0,11.9,99.0
1,14521010,Amtsberg,526,259,266,188,67,530,26,269,2105,25.0,12.3,12.6,8.9,3.2,25.2,1.2,12.8,98.8
2,14521020,"Annaberg-Buchholz, Stadt",3067,1533,1523,789,345,3052,110,1946,12255,25.0,12.5,12.4,6.4,2.8,24.9,0.9,15.9,99.1
3,14521035,"Aue-Bad Schlema, Stadt",3032,1512,1508,759,240,2989,134,2023,12063,25.1,12.5,12.5,6.3,2.0,24.8,1.1,16.8,98.9
4,14521040,Auerbach,424,220,216,110,26,445,14,286,1727,24.6,12.7,12.5,6.4,1.5,25.8,0.8,16.6,99.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,14730310,"Torgau, Stadt",2669,1342,1365,786,266,2677,119,1645,10750,24.8,12.5,12.7,7.3,2.5,24.9,1.1,15.3,98.9
415,14730320,Trossin,165,84,84,53,22,164,8,92,664,24.8,12.7,12.7,8.0,3.3,24.7,1.2,13.9,98.8
416,14730330,Wermsdorf,645,323,323,171,78,649,31,391,2580,25.0,12.5,12.5,6.6,3.0,25.2,1.2,15.2,98.8
417,14730340,Wiedemar,780,384,390,238,59,759,30,483,3093,25.2,12.4,12.6,7.7,1.9,24.5,1.0,15.6,99.0


## Statistik-DAten auf wide stellen, Ergebnisdaten mit Statistik-Daten verbinden

In [163]:
def statistic_long_to_wide(df): 
    
    df_wide = pd.pivot(df, index=["AGS", "Ort" ,"WKNr"], columns="Statistik", values=["Stimmen", "Anteil"])\
        .reset_index().fillna("0")
    # Multi-Index-Columns werden aufgelöst
    df_wide.columns = [" ".join(col).strip() for col in df_wide.columns.values]
    df_wide[["Stimmen Wahlberechtigte", "Stimmen Wähler"]] = \
        df_wide[["Stimmen Wahlberechtigte", "Stimmen Wähler"]].astype(int)
    
    return df_wide
    
df_daten_wide = statistic_long_to_wide(df_daten)

## Ergebnisse und Statistik mergen

In [164]:
def merge_results_and_stats(df_results, df_stats): 
    # Results mit Statistik mergen
    df_all_merge = pd.merge(df_results, df_stats, on=["AGS", "Ort"])
    return df_all_merge

df_final = merge_results_and_stats(df_results_correct, df_daten_wide)

In [165]:
df_final.columns

Index(['AGS', 'Ort', '2_Stimmen AfD', '2_Stimmen CDU', '2_Stimmen Die Linke',
       '2_Stimmen FDP', '2_Stimmen Grüne', '2_Stimmen SPD',
       '2_Stimmen Ungültige Stimmen', '2_Stimmen andere Parteien',
       '2_Stimmen gültige Stimmen', '2_Anteil AfD', '2_Anteil CDU',
       '2_Anteil Die Linke', '2_Anteil FDP', '2_Anteil Grüne', '2_Anteil SPD',
       '2_Anteil Ungültige Stimmen', '2_Anteil andere Parteien',
       '2_Anteil gültige Stimmen', 'WKNr', 'Stimmen Wahlberechtigte',
       'Stimmen Wähler', 'Anteil Wahlberechtigte', 'Anteil Wähler'],
      dtype='object')

## Historisches Ergebnis mit einrechnen


In [166]:
dfh = wahlergebnisse_2017.copy()
dfh["AGS"]= dfh["AGS"].astype(str)
dfh = dfh.loc[:, dfh.columns != "Ort"]


In [167]:
df_final = df_final.merge(dfh, on="AGS")

## Gewinner bestimmen (wichtig für Einfärbung der Karte)

In [168]:
df_final["2_Gewinner"] = df_final[['2_Stimmen AfD', '2_Stimmen CDU',
       '2_Stimmen FDP', '2_Stimmen Grüne', '2_Stimmen Die Linke', '2_Stimmen SPD']].idxmax(axis=1)
df_final["2_Gewinner"] = df_final["2_Gewinner"].str.split(" ")
df_final["2_Gewinner"] = df_final["2_Gewinner"].map(lambda x: x[1])



In [ ]:
# TODO: stimmen1 gewinner!!! 

In [169]:
df_final["2_Gewinner"]

0      AfD
1      SPD
2      AfD
3      SPD
4      AfD
      ... 
411    SPD
412    AfD
413    SPD
414    AfD
415    AfD
Name: 2_Gewinner, Length: 416, dtype: object

In [170]:
df_final.to_csv("wahlergebnisse_testdaten.csv", index=False)

In [172]:
df_final = df_final[0:0]

## Datawrapper-Grafik anlegen

In [173]:
def create_datawrapper(chart_id, df): 

    
    dw.add_data(chart_id, df)

    # UZ Wahlbeteiligung einbauen
    # Wahlbeteiligung und Quellen eintragen: 
    source_name = "<b style='color: grey'>Landeswahlleiter</b>"
    source_url = "https://wahlen.sachsen.de/bundestagswahl-2021.html"
    byline = "Christoph Knoop | Die Mehrwertmacher GmbH"
    
    unterzeile = f"Welche Partei konnte in welcher Stadt und Gemeinde punkten?<br>Alle <b>Zweitstimmergebnisse</b> auf einen Blick."
    dw.update_description(chart_id,source_name=source_name, source_url=source_url,\
                          byline=byline, intro=unterzeile)

    #string_date1 = f"Datenstand: {datetime.now().strftime('%d.%m.%Y, %H:%M')}."
    string_date1 = f"<b>Datenstand: {datenstand}</b>"
    string_date2 = f" Graugefärbte Kommunen sind noch nicht ausgezählt."
    string_final = string_date1 + string_date2
    props = {'annotate': {'notes': string_final}}
    dw.update_metadata(chart_id, props)
    dw.publish_chart(chart_id, display=False)
        
    print("Aktualisierung Gemeinden Zweistimmen Map abgeschlossen.")

create_datawrapper("r675i", df_final)

Chart updated!
Chart's metadata updated!
Aktualisierung Gemeinden Zweistimmen Map abgeschlossen.


## Datei als PNG ausspielen



In [176]:
chart_id = "r675i"
folder = "./data/png_gemeinden/"
file = f"zweitstimme_gemeinden_map.png"
path = folder + file
dw.export_chart(chart_id, width=768, zoom=3, border_width=30, filepath = path)
    
print("Zweitstimmen Wahkreise als png ausgespielt.")    

File exported at data\png_gemeinden\zweitstimme_gemeinden_map.png
Zweitstimmen Wahkreise als png ausgespielt.


## PNG an Server schicken


In [178]:
from ftplib import FTP 
import pysftp 
pw = "Fair67?sea" 
name = "christophknoop" 
hostname = "app.dmwm.de" 
hostname2 = "christophknoop@app.dmwm.de" 
hostname3 = "https://datasets.app.dmwm.de" 
# ftp = FTP(hostname) 
# ftp.login(user=name, passwd = pw) 
cnopts = pysftp.CnOpts() 
cnopts.hostkeys = None 

# Ort der hochzuladenen Dateien

#folder = "./data/png_wahlkreise"
url_png = "./data/png_gemeinden/zweitstimme_gemeinden_map.png"
#folder = r"C:\\Users\\c_knoop\\Desktop\\BTW_2021\\data\\png_gemeinden\\*.png"
with pysftp.Connection(hostname, username=name, password=pw, cnopts=cnopts) as sftp: 
    sftp.cwd('/var/www/christophknoop/wahlkarte/png/wahlkreise_zweitstimme') 
    sftp.put(url_png)  
               # upload file to public/ on remote     #sftp.put("./Daten/Abschlussdateien/nicht_kumuliert.csv")
sftp.close() 
print("PNGs Zweitstimme Gemeinden Map an Server gesendet.")

PNGs Zweitstimme Gemeinden Map an Server gesendet.


Grundsätzlich haben wir **sechs freie Parteien-Slots** und andere Parteien. Die sechs gesetzten Parteien möchten wir jetzt nach Stärke verteilen.

**Zunächst nutzen wir dabei nur die Zweitstimmen**, bei Erststimmen müssen wir nochmal schauen 


Dafür nutze ich die Daten **vor** dem long to wide. Die Long-Daten finden sich in diesen Dataframes: 

df_es_flat 

df_zs_flat

# ERSTSTIMME

In [160]:
# df_es 



,AGS,Ort,Partei,Stimmart,Stimmen,Anteil
0,14730010,Arzberg,Ungültige Stimmen,Erststimme,7,0.9
2,14730010,Arzberg,gültige Stimmen,Erststimme,793,99.1
4,14730010,Arzberg,AfD,Erststimme,198,25.0
6,14730010,Arzberg,CDU,Erststimme,98,12.4
8,14730010,Arzberg,LINKE,Erststimme,100,12.6
...,...,...,...,...,...,...
30,14612000,Dresden,SPD,Erststimme,65702,24.9
34,14612000,Dresden,Ungültige Stimmen,Erststimme,2691,1.0
38,14612000,Dresden,dieBasis,Erststimme,3009,1.1
40,14612000,Dresden,gültige Stimmen,Erststimme,263882,99.0


## Kandidaten für jede Partei und Wahlkreis herausfiltern 

In [60]:
df_k = pd.read_excel("./data/statistik-sachsen_bundestagswahl2021_direktbewerber.xlsx", header=1)

In [54]:
# # nur Kandidaten der sechs größeren Parteien behalten
# liste_parteien = ["AfD", "CDU", "DIE LINKE", "FDP", "GRÜNE", "SPD"]
# df_kandidaten = df_k[df_k["Wahlvorschlag \nKurzbezeichnung"].isin(liste_parteien)] 
# df_kandidaten["Full_Name"] = df_kandidaten['Name'] +" (" + df_kandidaten['Wahlvorschlag \nKurzbezeichnung']+")"

# df_kandidaten.rename(columns={"Wahlkreis 151 Nordsachsen":"Wahlkreis", "Nummer":"Listennummer", \
#                              "Wahlvorschlag \nKurzbezeichnung": "Partei", \
#                               "Wahlvorschlag \nLangbezeichnung": "Partei_lang", \
#                              "Name":"Name", "Full_Name": "Name_lang"}, inplace=True)
# # richtige Formate vergeben
# df_kandidaten["Wahlkreis"] = df_kandidaten["Wahlkreis"].astype(str)
# df_kandidaten["Listennummer"] = df_kandidaten["Listennummer"].astype(str)

<ipython-input-54-9ed62e74cfbf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kandidaten["Full_Name"] = df_kandidaten['Name'] +" (" + df_kandidaten['Wahlvorschlag \nKurzbezeichnung']+")"
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-54-9ed62e74cfbf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [78]:
#alle Kandidaten behalten 
df_kandidaten_all = df_k.copy()
def new_name(el):
    
    try:
        new_list = el.split(", ")
    
        new_name = new_list[1] + " " + new_list[0]
       
        return new_name
    except: 
        return el
df_kandidaten_all["Name"] = df_kandidaten_all["Name"].map(new_name)
df_kandidaten_all["Full_Name"] = df_kandidaten_all['Name'] +" (" + df_kandidaten_all['Wahlvorschlag \nKurzbezeichnung']+")"

df_kandidaten_all.rename(columns={"Wahlkreis 151 Nordsachsen":"KreisId", "Nummer":"Listennummer", \
                             "Wahlvorschlag \nKurzbezeichnung": "Partei", \
                              "Wahlvorschlag \nLangbezeichnung": "Partei_lang", \
                             "Name":"Name", "Full_Name": "Name_lang", "Nummer":"ParteiId"}, inplace=True)
# richtige Formate vergeben
#df_kandidaten_all["Wahlkreis"] = df_kandidaten_all["Wahlkreis"].astype(str)
#df_kandidaten_all["Listennummer"] = df_kandidaten_all["Listennummer"].astype(str)

KeyError: 'Wahlkreis'

In [80]:
df_kandidaten_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   KreisId      205 non-null    object
 1   ParteiId     205 non-null    object
 2   Partei       205 non-null    object
 3   Partei_lang  205 non-null    object
 4   Name         205 non-null    object
 5   Name_lang    205 non-null    object
dtypes: object(6)
memory usage: 9.7+ KB


## dict Übersetzung WK-Id WK-Name

In [81]:
wk_dict = {"151": "Nordsachsen", "152":"Leipzig I", "153": "Leipzig II", "154":"Leipziger Land", \
          "155": "Meißen", "156":"Bautzen I", "157":"Görlitz", "158":"SSOE", "159": "Dresden I", \
          "160": "Dresden II/Bautzen II", "161": "Mittelsachsen", "162":"Chemnitz", "163":"Chemnitzer Uml./Erzg. II", \
          "164": "Erzgebirgskreis I", "165":"Zwickau", "166":"Vogtlandkreis"}

## Ergebnisse und Daten mergen

In [82]:
df_es.KreisId = df_es.KreisId.astype(str)
df_es.ParteiId = df_es.ParteiId.astype(str)

df_kandidaten_all.KreisId = df_kandidaten_all.KreisId.astype(str)
df_kandidaten_all.ParteiId = df_kandidaten_all.ParteiId.astype(str)

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [83]:
df_all = pd.merge(df_es, df_kandidaten_all[["KreisId", "ParteiId", "Name_lang", "Name"]], \
                     on=["KreisId", "ParteiId"])

In [84]:
df_all

,AGS,KreisId,Ort,Partei,ParteiId,Stimmart,Stimmen,Anteil,Name_lang,Name
0,14730010,151,Arzberg,AfD,1,Erststimme,198,25.0,René Bochmann (AfD),René Bochmann
1,14730020,151,"Bad Düben, Stadt",AfD,1,Erststimme,1110,25.2,René Bochmann (AfD),René Bochmann
2,14730030,151,Beilrode,AfD,1,Erststimme,568,25.3,René Bochmann (AfD),René Bochmann
3,14730045,151,"Belgern-Schildau, Stadt",AfD,1,Erststimme,932,25.0,René Bochmann (AfD),René Bochmann
4,14730050,151,Cavertitz,AfD,1,Erststimme,283,24.5,René Bochmann (AfD),René Bochmann
...,...,...,...,...,...,...,...,...,...,...
4008,14523420,166,Tirpersdorf,III. Weg,17,Erststimme,69,6.6,Udo Sieghart (III. Weg),Udo Sieghart
4009,14523430,166,"Treuen, Stadt",III. Weg,17,Erststimme,28,0.6,Udo Sieghart (III. Weg),Udo Sieghart
4010,14523440,166,Triebel/Vogtl.,III. Weg,17,Erststimme,0,0.0,Udo Sieghart (III. Weg),Udo Sieghart
4011,14523450,166,Weischlitz,III. Weg,17,Erststimme,0,0.0,Udo Sieghart (III. Weg),Udo Sieghart


## Gewinner/Verlierer sortieren

In [85]:
dict_colors = {"AfD":"#009ee0", "CDU":"#32302e", "SPD":"#e3000f", "DIE LINKE":"#ab006d", \
               "GRÜNE":"#a0c864", "FDP":"#fccc00", "FREIE WÄHLER":"#f39401"}



In [86]:
def create_erststimme_datawrapper(df): 
    
    cols = ["AGS", "Ort", "KreisId", "KreisName", "Gewinner_Partei", "Gewinner_Name", "Gewinner_c", \
            "Partei_1", "Partei_1_k", "Partei_1_c", "Partei_1_a", \
           "Partei_2", "Partei_2_k", "Partei_2_c", "Partei_2_a",
           "Partei_3", "Partei_3_k", "Partei_3_c", "Partei_3_a",
           "fließtext_rest"]
    
    new_df = pd.DataFrame(columns=cols)
    
    
    
    
    
    all_ags = df.AGS.unique()
    for ags in all_ags:
        
        
        
        df_ = df[df.AGS==ags]
        ort = df_["Ort"].unique()[0]
        kreis = df_["KreisId"].unique()[0]
        kreisname = wk_dict[kreis]
        no_parties = ["Ungültige Stimmen", "gültige Stimmen"]
        df_reduce = df_[~df_.Partei.isin(no_parties)]
        
        df_sort = df_reduce.sort_values(by="Anteil", ascending=False)
        
         # HELPER get color
        def get_color(partei): 
            try: 
                return dict_colors[partei]
            except: 
                return "#b5152b"
        
        # Gewinner
        gewinner_partei = df_sort.iloc[0].Partei
        gewinner_name = df_sort.iloc[0].Name
        gewinner_color = get_color(gewinner_partei)
        
       
        
        # ersten drei Kandidaten
        partei1 = df_sort.iloc[0].Partei
        partei1_k = df_sort.iloc[0].Name
        partei1_c = get_color(partei1)
        partei1_a = df_sort.iloc[0].Anteil
        
         # ersten drei Kandidaten
        partei2 = df_sort.iloc[1].Partei
        partei2_k = df_sort.iloc[1].Name
        partei2_c = get_color(partei2)
        partei2_a = df_sort.iloc[1].Anteil
        
         # ersten drei Kandidaten
        partei3 = df_sort.iloc[2].Partei
        partei3_k = df_sort.iloc[2].Name
        partei3_c = get_color(partei3)
        partei3_a = df_sort.iloc[2].Anteil
        
        long_string = ""
        
        for i in range(2, len(df_sort)):
        
            party_string = f" +++ {df_sort.iloc[i].Name_lang}, {df_sort.iloc[i].Anteil}%"
            long_string = long_string + party_string
    
       
       
        
        
        data_ser = [ags, ort, kreis, kreisname, gewinner_partei, gewinner_name, gewinner_color, \
                   partei1, partei1_k, partei1_c, partei1_a, \
                   partei2, partei2_k, partei2_c, partei2_a, \
                   partei3, partei3_k, partei3_c, partei3_a, \
                   long_string]
        
        new_ser = pd.Series(data_ser, index=cols)
        
        new_df = new_df.append(new_ser, ignore_index=True)
    
    return new_df
    
df_es_dw = create_erststimme_datawrapper(df_all)

## Grafik aktualisieren

In [87]:
df_es_dw

,AGS,Ort,KreisId,KreisName,Gewinner_Partei,Gewinner_Name,Gewinner_c,Partei_1,Partei_1_k,Partei_1_c,Partei_1_a,Partei_2,Partei_2_k,Partei_2_c,Partei_2_a,Partei_3,Partei_3_k,Partei_3_c,Partei_3_a,fließtext_rest
0,14730010,Arzberg,151,Nordsachsen,AfD,René Bochmann,#009ee0,AfD,René Bochmann,#009ee0,25.0,SPD,Rüdiger Kleinke,#e3000f,24.2,Die Partei,Karsten Gutjahr,#b5152b,16.8,"+++ Karsten Gutjahr (Die PARTEI), 16.8% +++ P..."
1,14730020,"Bad Düben, Stadt",151,Nordsachsen,AfD,René Bochmann,#009ee0,AfD,René Bochmann,#009ee0,25.2,SPD,Rüdiger Kleinke,#e3000f,25.0,CDU,Dr. Christiane Schenderlein,#32302e,12.5,"+++ Dr. Christiane Schenderlein (CDU), 12.5% ..."
2,14730030,Beilrode,151,Nordsachsen,AfD,René Bochmann,#009ee0,AfD,René Bochmann,#009ee0,25.3,SPD,Rüdiger Kleinke,#e3000f,25.2,CDU,Dr. Christiane Schenderlein,#32302e,12.7,"+++ Dr. Christiane Schenderlein (CDU), 12.7% ..."
3,14730045,"Belgern-Schildau, Stadt",151,Nordsachsen,SPD,Rüdiger Kleinke,#e3000f,SPD,Rüdiger Kleinke,#e3000f,25.1,AfD,René Bochmann,#009ee0,25.0,Die Partei,Karsten Gutjahr,#b5152b,14.2,"+++ Karsten Gutjahr (Die PARTEI), 14.2% +++ D..."
4,14730050,Cavertitz,151,Nordsachsen,SPD,Rüdiger Kleinke,#e3000f,SPD,Rüdiger Kleinke,#e3000f,24.7,AfD,René Bochmann,#009ee0,24.5,Die Partei,Karsten Gutjahr,#b5152b,15.1,"+++ Karsten Gutjahr (Die PARTEI), 15.1% +++ D..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,14523420,Tirpersdorf,166,Vogtlandkreis,AfD,Mathias Weiser,#009ee0,AfD,Mathias Weiser,#009ee0,24.2,SPD,Kay Burmeister,#e3000f,24.1,CDU,Yvonne Magwas,#32302e,12.7,"+++ Yvonne Magwas (CDU), 12.7% +++ Johannes H..."
417,14523430,"Treuen, Stadt",166,Vogtlandkreis,AfD,Mathias Weiser,#009ee0,AfD,Mathias Weiser,#009ee0,25.2,SPD,Kay Burmeister,#e3000f,25.0,ÖDP,Dietmar Eichhorn,#b5152b,13.1,"+++ Dietmar Eichhorn (ÖDP), 13.1% +++ Yvonne ..."
418,14523440,Triebel/Vogtl.,166,Vogtlandkreis,SPD,Kay Burmeister,#e3000f,SPD,Kay Burmeister,#e3000f,25.5,AfD,Mathias Weiser,#009ee0,24.8,ÖDP,Dietmar Eichhorn,#b5152b,13.2,"+++ Dietmar Eichhorn (ÖDP), 13.2% +++ Yvonne ..."
419,14523450,Weischlitz,166,Vogtlandkreis,AfD,Mathias Weiser,#009ee0,AfD,Mathias Weiser,#009ee0,24.9,SPD,Kay Burmeister,#e3000f,24.6,Die Linke,Johannes Höfer,#b5152b,12.6,"+++ Johannes Höfer (DIE LINKE), 12.6% +++ Yvo..."


In [93]:

# chart_id = "pLIgz"
#testid
#chart_id = "i8QRf"

#test2
chart_id = "5OhSu"
dw.add_data(chart_id, df_es_dw)

# Wahlbeteiligung und Quellen eintragen: 
source_name = "<b style='color: grey'>Landeswahlleiter</b>"
source_url = "https://wahlen.sachsen.de/bundestagswahl-2021.html"
byline = "Christoph Knoop | Die Mehrwertmacher GmbH"

unterzeile = f"<b>Ergebnis Erststimmen für Direktkandidaten</b><br>Graugefärbte Gemeinden noch ohne Ergebnis."
dw.update_description(chart_id,source_name=source_name, source_url=source_url,\
                      byline=byline, intro=unterzeile)

# Datenstand und Auszählungsstand eintragen
string_1 = "<b>Ergebnisse auf Gemeindeeben. Kandidaten, die in einer Gemeinde gewonnen haben, können trotzdem im gesamten Wahlkreis unterlegen sein.</b><br>"
string_date1 = f"Datenstand: {datenstand}."
#string_date1 = datenstand
#string_date2 = f" <b>{abfrage} ausgezählt.</b>"
string_final = string_1 + string_date1
props = {'annotate': {'notes': string_final}}
dw.update_metadata(chart_id, props)
dw.publish_chart(chart_id, display=False)
#endregion
print("Aktualisierung Erststimme Gemeindekarte abgeschlossen.")

Chart updated!
Chart's metadata updated!
Aktualisierung Erststimme Gemeindekarte abgeschlossen.


In [92]:
df_es_dw[df_es_dw["Ort"].str.contains("Dresden")]

,AGS,Ort,KreisId,KreisName,Gewinner_Partei,Gewinner_Name,Gewinner_c,Partei_1,Partei_1_k,Partei_1_c,Partei_1_a,Partei_2,Partei_2_k,Partei_2_c,Partei_2_a,Partei_3,Partei_3_k,Partei_3_c,Partei_3_a,fließtext_rest
231,146120001,Dresden I,159,Dresden I,AfD,Jens Maier,#009ee0,AfD,Jens Maier,#009ee0,25.0,SPD,Rasha Nasr,#e3000f,24.9,Die Linke,Katja Kipping,#b5152b,12.5,"+++ Katja Kipping (DIE LINKE), 12.5% +++ Dr. ..."
232,146120002,Dresden II,160,Dresden II/Bautzen II,SPD,Stephan Schumann,#e3000f,SPD,Stephan Schumann,#e3000f,24.9,AfD,Andreas Harlaß,#009ee0,24.8,Die Linke,Silvio Lang,#b5152b,12.5,"+++ Silvio Lang (DIE LINKE), 12.5% +++ Lars R..."


## Grafik als Png ausspielen

## PNG an Server schicken